In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
P3 = Place(model, 'P3')

T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')
T3 = Transition(model, 'T3')

jobclass = ClosedClass(model, 'Class1', 4, P1, 0)

In [3]:
# T1 - Multiple firing modes
# Mode1: requires 2 tokens from P1, produces 2 tokens to P2
mode1 = T1.add_mode('Mode1')
T1.setDistribution(mode1, Exp(2))
T1.setEnablingConditions(mode1, jobclass, P1, 2)
T1.setFiringOutcome(mode1, jobclass, P2, 2)

# Mode2: requires 1 token from P1, produces 1 token to P3
mode2 = T1.add_mode('Mode2')
T1.setDistribution(mode2, Exp(1))
T1.setEnablingConditions(mode2, jobclass, P1, 1)
T1.setFiringOutcome(mode2, jobclass, P3, 1)

# T2 - Standard transition: P2 -> P1
mode3 = T2.add_mode('Mode3')
T2.setDistribution(mode3, Exp(4))
T2.setEnablingConditions(mode3, jobclass, P2, 1)
T2.setFiringOutcome(mode3, jobclass, P1, 1)

# T3 - Transition with inhibiting condition
# Requires 3 tokens from P3, but inhibited when P2 has tokens
mode4 = T3.add_mode('Mode4')
T3.setDistribution(mode4, Exp(1))
T3.setEnablingConditions(mode4, jobclass, P3, 3)
T3.setInhibitingConditions(mode4, jobclass, P2, 1)  # Inhibited by P2
T3.setFiringOutcome(mode4, jobclass, P1, 3)

In [4]:
# Set up routing matrix
routingMatrix = model.init_routing_matrix()

# Place to transition routing
routingMatrix.set(jobclass, jobclass, P1, T1, 1.0)  # P1 -> T1 (both modes)
routingMatrix.set(jobclass, jobclass, P2, T2, 1.0)  # P2 -> T2
routingMatrix.set(jobclass, jobclass, P2, T3, 1.0)  # P2 -> T3 (inhibiting)
routingMatrix.set(jobclass, jobclass, P3, T3, 1.0)  # P3 -> T3

# Transition to place routing
routingMatrix.set(jobclass, jobclass, T1, P2, 1.0)  # T1 -> P2 (Mode1)
routingMatrix.set(jobclass, jobclass, T1, P3, 1.0)  # T1 -> P3 (Mode2)
routingMatrix.set(jobclass, jobclass, T2, P1, 1.0)  # T2 -> P1
routingMatrix.set(jobclass, jobclass, T3, P1, 1.0)  # T3 -> P1

model.link(routingMatrix)

In [5]:
# Set initial state - all 4 tokens start in P1
P1.setState([jobclass.getPopulation()])
P2.setState([0])
P3.setState([0])

In [6]:
# Solve the model
solver = SolverJMT(model, seed=23000)
avgTable = solver.get_avg_table()

JMT Model: /tmp/workspace/jsim/11064491680299168389/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.739824s.
  Station JobClass    QLen  Util   RespT  ResidT    ArvR    Tput
0      P1   Class1  1.3667   0.0  0.5680  0.5680  2.4787  2.4778
1      P2   Class1  0.7450   0.0  0.4371  0.1093  1.7363  1.7134
2      P3   Class1  1.8403   0.0  2.6351  1.3176  0.7158  0.7163


In [7]:
# Solve the model
solver = SolverJMT(model, seed=23000)
avgTable = solver.get_avg_table()

JMT Model: /tmp/workspace/jsim/4381402928643889708/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.952547s.
  Station JobClass    QLen  Util   RespT  ResidT    ArvR    Tput
0      P1   Class1  1.3667   0.0  0.5680  0.5680  2.4787  2.4778
1      P2   Class1  0.7450   0.0  0.4371  0.1093  1.7363  1.7134
2      P3   Class1  1.8403   0.0  2.6351  1.3176  0.7158  0.7163
